# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [8]:
!git clone https://github.com/KU-HIAI/tomok.git

fatal: destination path 'tomok' already exists and is not an empty directory.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [9]:
cd tomok

/content/tomok/tomok


## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [10]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS241421_04020303_05)
class KDS241421_04020303_05 (tomok.RuleUnit):

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Seonghan Yoon'  # 작성자명
    ref_code = 'KDS 24 14 21 4.2.3.3 (5)' # 건설기준문서
    ref_date = '2021-04-12'  # 디지털 건설문서 작성일
    doc_date = '2023-11-14'  # 건설기준문서->디지털 건설기준 변환 기준일
    title = '2축 응력 상태의 거더 복부의 유효인장강도'    # 건설기준명

    #
    description = """
    콘크리트교 설계기준 (한계상태설계법)
    4. 설계
    4.2 사용한계상태
    4.2.3 균열
    4.2.3.3 간접 균열 제어
    (5)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """
    """
    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
    subgraph Python_Class
    A["거더 복부의 유효인장강도"];
    B["KDS 24 14 21 4.2.3.3 (5)"];
    A ~~~ B
    end

		subgraph Variable_def;
		VarIn1[/입력변수: 주압축응력/];
		VarIn2[/입력변수: 콘크리트의 기준압축강도/];
		VarIn3[/입력변수: 콘크리트 하위 0.5 분위 기준인장강도/];
		VarOut1[/출력변수: 거더 복부의 유효인장강도/];
		VarOut1~~~VarIn1 & VarIn2  & VarIn3

		end
		Python_Class ~~~ Variable_def;
		Variable_def--->D--->E--->F
		D["<img src='https://latex.codecogs.com/svg.image?f_{2}\leq&space;0.6f_{ck}'>---------------------------------"]
		E["<img src='https://latex.codecogs.com/svg.image?f_{cte}=(1-0.8\frac{f_2}{f_{ck}})f_{ctk}'>---------------------------------"]

		F(["거더 복부의 유효인장강도"])
    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정,
    @tomok.rule_method
    def Effective_tensile_strength_of_girderweb(fOfcte,fIf2,fIfck,fIfctk) -> float:
        """2축 응력 상태의 거더 복부의 유효인장강도

        Args:
             fOfcte (float): 거더 복부의 유효인장강도
             fIf2 (float): 주압축응력
             fIfck (float): 콘크리트의 기준압축강도
             fIfctk (float): 콘크리트 하위 0.5 분위 기준인장강도




        Returns:
            float: 콘크리트교 설계기준 (한계상태설계법) 4.2.3.3 간접 균열 제어 (5)의 값
        """

        fIf2 = min(fIf2, 0.6*fIfck)
        fOfcte = (1-0.8*fIf2/fIfck)*fIfctk

        return fOfcte


작성한 룰 유닛은 아래의 코드 블럭과 같이 생성하여, 작성자가 임의로 검증을 수행할 수 있습니다.

In [11]:
my_RuleUnit = KDS241421_04020303_05()

In [12]:
fOfcte = None
fIf2 = 28
fIfck = 24
fIfctk = 40

In [13]:
Rule_Review_Result = my_RuleUnit.Effective_tensile_strength_of_girderweb(fOfcte,fIf2,fIfck,fIfctk)
# 해당건설기준 항목 의 값은?

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: 20.8


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [14]:
my_RuleUnit.render_markdown()